In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
#Installing PySpark package

! pip install pyspark

In [ ]:
from pyspark.sql import SparkSession

spark = SparkSession\
        .builder\
        .appName("Python Spark regression example")\
        .config("spark.some.config.option", "some-value")\
        .getOrCreate()

# **Creating a spark Dataframe, not to be confused with Pandas Dataframe**
* format can take values like csv, parquet, json etc..
* inferSchema - deduces schema from the available data
* Schema has a StructType that contains many StructFields. Each StructFiels has a name, data type, and Boolean flag to designate whether it accepts null values or not


In [ ]:


df = spark.read.format("csv").\
                options(inferSchema = True, header = True).\
                load("../input/advertisingcsv/Advertising.csv", header=True)


print(df.show(5, False)) # Shows first five records and does not truncate values

print(df.printSchema())

In [ ]:
# Describe the dataset, the result will be shown for all feature columns
df.describe().show()

# Describe the numeriacl statistics for just a signle colums

df.select("TV").describe().show()

In [ ]:
# Printitng the column names of the dataframe
col_names = df.columns
print(col_names)

In [ ]:
#Renaming the column "_c0" to "Index" by using withcolumnRenamed method

df = df.withColumnRenamed("_c0", "Index")

# Printing to show the change in column name
df.show(5)

# Feature Vector can be obtained using different options. Below, we will see three such options to do it

## Creating the feature vector using high level RFormula
* (~) ---> seperates target labels and individual features
* (.) ---> All features except the target column
* (+) ---> Concat the indicated terms
  
    
### We define the formula and call fit & transform on the dataset to yield required features column
### In the section below, we have separated target term and feature terms. And created a features vector containg values from TV, Newspaper and Radio columns
### It is mandatory that both the features vector and labels are represeted as double while using Spark ML 

In [ ]:
from pyspark.ml.feature import RFormula

supervised = RFormula(formula = "Sales ~  TV + Radio + Newspaper")
fittedRF = supervised.fit(df)
preparedDF = fittedRF.transform(df)
preparedDF.show(5, False)

In [ ]:
# Same above transformation can be obtained ina sig

testDf = RFormula(formula = "Sales ~  TV + Radio + Newspaper").fit(df).transform(df)

testDf.show(5, False)

## Using Vector Assembler to create Dense Feature Vector

### It Accepts the name of input columns to be considered for assembling via "setInputCols"
### Gives a features column that is added to the dataframe

In [ ]:
from pyspark.ml.feature import VectorAssembler

va = VectorAssembler()\
    .setInputCols(["TV","Radio", "Newspaper"])\
    .setOutputCol("va_features")

va.transform(preparedDF).show(5)

##  Using RDDs to create feature vector and label

In [ ]:
# We are defining a functions that takes in a dataframe and converts it to RDD
# applies a Map function
# The function passed onto Map function returs All the columns except first & last cloumn to prepare the feature vector
# and only the last columns ("Sales") as the label vector

from pyspark.ml.linalg import Vectors

def vecTransform (data):
    return data.rdd.map(lambda r: [Vectors.dense(r[:-1]),r[-1]]).toDF(['features','label'])



vecTransform(df).show(5, False)

## Using StandardScaler to scale the data

In [ ]:

from pyspark.ml.feature import StandardScaler


scaler = StandardScaler().setInputCol("features").setOutputCol("scaledFeatures")

scaledDF = scaler.fit(preparedDF).transform(preparedDF)

In [ ]:
scaledDF.show(5, False)

In [ ]:
preparedDF.show(5)

# Using VectorIndexer to Index feature Vector
- Vector indexer takes in a column of Vector and return an indexed Vector column as output
- This helps in identifying any categorical data present within the feature vector
- maXCategories help in identifying the categorical that if any present in the feature vector. If the set of values are greater than maxCategories, then the values are treated as continuous


In [ ]:
from pyspark.ml import Pipeline
from pyspark.ml.regression import LinearRegression
from pyspark.ml.feature import VectorIndexer
from pyspark.ml.evaluation import RegressionEvaluator

featureIndexer = VectorIndexer().setInputCol("features").setOutputCol("indexedFeatures").setMaxCategories(4).fit(preparedDF)

data = featureIndexer.transform(preparedDF)

data.show(5)

In [ ]:
# Scaling dataset

scaler = StandardScaler().setInputCol("indexedFeatures").setOutputCol("scaledFeatures")

finalScaledData = scaler.fit(data).transform(data)

finalScaledData.show(5, False)

In [ ]:
inputData = finalScaledData.select(["scaledFeatures", "label"])

# inputData = inputData.withColumnRenamed("scaledFeatures", "features")

inputData.show(5, False)

# Fitting a Linear Regression Model

In [ ]:
# Spliting dataset into training and test set

training, test = inputData.randomSplit([0.8,0.2])

print("Training set has {} records".format(training.count()))
print("Training set has {} records".format(test.count()))

In [ ]:
# Creating an instace of LinearRegression class passing on parameters

lin_reg = LinearRegression(featuresCol="scaledFeatures", labelCol = "label").setMaxIter(10).setRegParam(0.3).setElasticNetParam(0.8)

print(lin_reg.explainParams()) # Explains the parameters of the model

model = lin_reg.fit(training) # Fitting the Model



In [ ]:
print('###' * 30)
print("Coffiecients of the model are: {}".format(model.coefficients))
print('###' * 30)
print('###' * 30)

print("Intercept of linear regression equation is: {}".format(model.intercept))

print('###' * 30)
print('###' * 30)
summary = model.summary # creating an instance of model summary

print('###' * 30)
print('###' * 30)
print("The residual values after the fit are: ")
summary.residuals.show() # This returns only the first 20 values

print('###' * 30)
print('###' * 30)
print("The calculated loss for the objective function at each iteration is: ")
print(summary.objectiveHistory)

print('###' * 30)
print('###' * 30)
print("The RMSE value of the fitted model: {}".format(summary.rootMeanSquaredError))

print('###' * 30)
print('###' * 30)
print("The R2 score of the fitted model: {}".format(summary.r2))


# Predictions on Test Data

In [ ]:
predictions = model.transform(test)
predictions.show(30, False)

In [ ]:
from pyspark.ml.evaluation import RegressionEvaluator

reg_eval = RegressionEvaluator(predictionCol = "prediction", labelCol = "label", metricName = "rmse")

print("The RMSE value on test data: {}".format(reg_eval.evaluate(predictions)))



# Deifining a ML Pipeline

1. We ingest data and create a Dataframe - df
2. The dataset is spli into Train and Test set
3. Instantiate a Vector assemble to create Feature Vector
4. Use Vector Indexer to automatically index if there are any categorical / discrete values
5. We scale this Feature Vector created using Vector Indexer
6. And Finally, instantiate a Linear Regression model

In [ ]:
from pyspark.ml import Pipeline # Importing Pipeline


train, test = df.randomSplit([0.8, 0.2]) # splitting datatset into train and test set
assembler = VectorAssembler(inputCols = ["TV", "Radio","Newspaper"], outputCol ="features")
indexer = VectorIndexer(inputCol ="features", outputCol = "indexed_features")
scaler = StandardScaler(inputCol = "indexed_features", outputCol = "scaledFeatures")
lr = LinearRegression(featuresCol = "scaledFeatures", labelCol = "Sales")

pipeline = Pipeline(stages = [assembler, indexer, scaler, lr])

model = pipeline.fit(train)
predictions = model.transform(test)

In [ ]:
predictions.show(10, False)